<a href="https://colab.research.google.com/github/serch86/CouponPonpare/blob/master/Mercado_Libre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mercado Libre challenge

In [1]:
# si estamos trabajando en colab, primero debemos
# instalar spark con las siguientes lineas de código

# En caso de que no se pueda instalar spark, revisar las lineas comentadas
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz  # Revisar la ultima version en https://downloads.apache.org/spark/ y modificar el código a esa linea
!tar xf spark-3.1.2-bin-hadoop2.7.tgz                                                  # Hacer modificaciones de la versión de spark en está linea también
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"                        # Hacer modificaciones de la versión de spark en está linea también
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos paquetería que será usada en el proyecto

In [68]:
import gzip
import pandas as pd
import re

import pyspark.sql.functions as F

from pyspark.sql.types import StringType, DoubleType, IntegerType, ArrayType, DateType

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, MinMaxScaler

from pyspark.sql.window import Window

### Submission example

In [6]:
# Rutas Allan
path_sample_submission = '/content/drive/MyDrive/analisis/reto_mercado_libre/Data/sample_submission.csv.gz'
path_test_data = '/content/drive/MyDrive/analisis/reto_mercado_libre/Data/test_data.csv'
path_training_data = '/content/drive/MyDrive/analisis/reto_mercado_libre/Data/train_data.parquet'
path_metadata = '/content/drive/MyDrive/analisis/reto_mercado_libre/Data/items_static_metadata_full.jl'
# Rutas Sergio
path_sample_submission = '/content/drive/MyDrive/analisis/mercado_libre/sample_submission.csv.gz'
path_test_data = '/content/drive/MyDrive/analisis/mercado_libre/test_data.csv'
path_training_data = '/content/drive/MyDrive/analisis/mercado_libre/train_data.parquet'
path_metadata = '/content/drive/MyDrive/analisis/mercado_libre/items_static_metadata_full.jl'

Cargamos los datos

In [7]:
# submission example
f=gzip.open(path_sample_submission,'rb')
ejemplo=pd.read_csv(f, header=None)

In [8]:
ejemplo.shape

(551472, 30)

### Test data

In [9]:
# test data
test = pd.read_csv(path_test_data)

In [10]:
test.shape

(551472, 2)

In [11]:
test.head(3)

,sku,target_stock
0,464801,3
1,645793,4
2,99516,8


In [12]:
# metadata
metadata = pd.read_json(path_metadata, lines=True)

In [13]:
metadata.head()

,item_domain_id,item_id,item_title,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,0,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,1,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,2,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,3,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,4,None,None


### Training data

In [42]:
df = spark.read.parquet(path_training_data)

In [43]:
df = df.withColumn('date',F.to_date(F.col('date')))

In [44]:
df.count(), len(df.columns)

(37660279, 9)

In [45]:
df.show(3)

+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|   sku|      date|sold_quantity|current_price|currency|listing_type|shipping_logistic_type|shipping_payment|minutes_active|
+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|464801|2021-02-01|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-02|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-03|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
only showing top 3 rows



In [46]:
df.select(F.col('sku')).dropDuplicates().count()

660916

In [47]:
df.groupBy(F.col('minutes_active')).count().show()

+------------------+-----+
|    minutes_active|count|
+------------------+-----+
|238.81278333333333|    1|
|         165.93155|    1|
|          763.2572|    1|
|        1117.57335|    1|
|1092.3185333333333|    1|
| 932.9090166666667|    1|
|1067.3185833333334|    1|
|1119.0794666666666|    1|
| 94.00356666666666|    1|
|1054.4643833333334|    1|
|1429.7637666666667|    1|
|         771.24885|    1|
|         614.97915|    1|
|         496.95435|    1|
| 649.4081666666667|    1|
|1233.6563833333335|    1|
|1178.1166333333335|    1|
|1419.6755666666668|    1|
| 829.4183833333334|    1|
| 814.0651499999999|    1|
+------------------+-----+
only showing top 20 rows



La variable objetivo hay que generarla.

Tal que la Probabilidad que el producto se agote en funcion del numero de unidades vendidas durante 1 mes es nuestra variable objetivo.

![mercadolibre.JPG](https://drive.google.com/uc?id=1abiIarCY5FRgLMlpyDfGy6oM5pmeYJJb)

Para ello hay que crear variables auxiliares como el mes normalizado y el mes acumulado, donde:

  - mes_normalizado es la suma de unidades vendidas durante el mes1 y mes2 en el dia n-esimo.
  - mes_acumulado la suma acumulada de unidades vendidas durante el mes_normalizado.

In [48]:
df.show()

+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|   sku|      date|sold_quantity|current_price|currency|listing_type|shipping_logistic_type|shipping_payment|minutes_active|
+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|464801|2021-02-01|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-02|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-03|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-04|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-05|            1|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|


## Feature Engineering

### Tratamiento de variables de fechas

Creamos las variables de dia y dia de la semana.

### Tratamiento variables categoricas

Pasemos a variables dummies las variables categoricas siguientes.

- day
- day of week
- currency
- listing_type
- shipping_logistic_type
- shipping_payment

Generando 49 columnas.

In [54]:
df = df.withColumn('dia', F.dayofmonth('date').cast('string'))

In [55]:
df = df.withColumn('dia_semana', F.date_format('date', 'EEEE'))

In [59]:
def remove_nulls(df):
    df_no_nulls = df

    for element in df_no_nulls.columns:
        if df_no_nulls.where(df_no_nulls[element].isNull()).count() != 0:
            print('\tThe column "{}" has null values'.format(element))
            df_no_nulls = df_no_nulls.where(df_no_nulls[element].isNotNull())
        if df_no_nulls.where(df_no_nulls[element].isNull()).count() == 0:
            print('The column "{}" does not have null values'.format(element))
            
    return df_no_nulls
    
def check_nulls(df):
    
    existing_nulls = False
    
    for element in df.columns:
        if df.where(df[element].isNull()).count() != 0:
            print('\tThe column "{}" has null values'.format(element))
            existing_nulls = True
            break
        if df.where(df[element].isNull()).count() == 0:
            print('The column "{}" does not have null values'.format(element))
            
    return existing_nulls


def get_dummies_spark(df, dummy_cols):   
     
    # check nulls
    
    existing_nulls = check_nulls(df)
    
    if existing_nulls == False:
            
        # StringIndexer + OneHotEncoder

        dictionaries = []

        for element in dummy_cols:
            print("StringIndexer + OneHotEncoder for column ", element)

            string_indexer = StringIndexer(inputCol=element, outputCol=element+'_category')
            onehotencoder = OneHotEncoder(dropLast=False, inputCol= string_indexer.getOutputCol(), outputCol=element+'_dummy')
            pipeline = Pipeline(stages=[string_indexer, onehotencoder])
            pipeline_model = pipeline.fit(df)
            dictionaries.append((element, list(enumerate(pipeline_model.stages[0].labels)), pipeline_model.stages[0]))
            df = pipeline_model.transform(df)
            df = df.drop(string_indexer.getOutputCol())

        # divide OneHotEncoder output in different columns

        for element in dictionaries:
            print("Divide OneHotEncoder output in several columns for original column ", element[0])

            df = (df.withColumn('activated_indices'+element[0], F.udf(lambda x: x.toArray().tolist(), ArrayType(DoubleType()))
                                (F.col(element[0]+'_dummy'))))

            vocab = [re.sub(r'\W', '_', value) for value in element[-1].labels]
            df = df.select(df.columns + [F.col("activated_indices"+element[0])[i] for i in range(len(vocab))])

            dictionary = {"activated_indices"+element[0]+"[{0}]".format(x): element[0]+'_'+vocab[x] for x in range(len(vocab))}

            # rename columns
            df = df.selectExpr(["{0} as {1}".format(x, x) if x not in dictionary else "{0} as {1}".format(x, dictionary[x]) 
                                          for x in df.columns])
            df = df.drop('activated_indices'+element[0], element[0]+'_dummy')
    
    else:
        print("There are nulls in your dataframe, please remove them or fill them before creating dummy features")
        
        
    return df

In [60]:
# get dummie cols
dummie_cols = [col for col,t in df.dtypes if t in ('string','boolean')]
dummie_cols

['currency',
 'listing_type',
 'shipping_logistic_type',
 'shipping_payment',
 'dia',
 'dia_semana']

In [61]:
df = get_dummies_spark(df,dummie_cols)

The column "sku" does not have null values
The column "date" does not have null values
The column "sold_quantity" does not have null values
The column "current_price" does not have null values
The column "currency" does not have null values
The column "listing_type" does not have null values
The column "shipping_logistic_type" does not have null values
The column "shipping_payment" does not have null values
The column "minutes_active" does not have null values
The column "dia" does not have null values
The column "dia_semana" does not have null values
StringIndexer + OneHotEncoder for column  currency
StringIndexer + OneHotEncoder for column  listing_type
StringIndexer + OneHotEncoder for column  shipping_logistic_type
StringIndexer + OneHotEncoder for column  shipping_payment
StringIndexer + OneHotEncoder for column  dia
StringIndexer + OneHotEncoder for column  dia_semana
Divide OneHotEncoder output in several columns for original column  currency
Divide OneHotEncoder output in sever

Variable lag de precio.

Hipotesis: El cambio en el precio afecta el numero de ventas. Si el precio disminuye en un cierto porcentaje, vease como oferta, es probable que la venta de articulos aumente y por ende se acabe antes el producto de lo esperado

Por otro lado, se estima que el cambio sera similar al actual *pero para evitar el data leakage se calcula el cambio con respecto al lag $t-1$, donde el lag es el precio del producto en el tiempo $t-n$* (Por ahora no se hace, se hace como la ecuacion)
$$\triangle p_{i,t} = p_{i,t} -  p_{i,t-1}$$

In [79]:
df = df.withColumn('price_lag_1',F.lag(df['current_price'], offset=1)\
          .over(Window.partitionBy("sku").orderBy('date')))

df = df.withColumn('current_price_diff',F.col('current_price') - F.col('price_lag_1'))
df = df.withColumn('current_price_div',F.col('current_price_diff')/F.col('current_price'))

In [81]:
df.select('sku','date','current_price','price_lag_1', 
          'current_price_diff','current_price_div').show()

+---+----------+-------------+-----------+--------------------+--------------------+
|sku|      date|current_price|price_lag_1|  current_price_diff|   current_price_div|
+---+----------+-------------+-----------+--------------------+--------------------+
| 26|2021-02-01|          6.9|       null|                null|                null|
| 26|2021-02-02|          6.9|        6.9|                 0.0|                 0.0|
| 26|2021-02-03|          6.9|        6.9|                 0.0|                 0.0|
| 26|2021-02-04|          6.5|        6.9|-0.40000000000000036|-0.06153846153846159|
| 26|2021-02-05|          6.5|        6.5|                 0.0|                 0.0|
| 26|2021-02-06|          6.5|        6.5|                 0.0|                 0.0|
| 26|2021-02-07|          6.5|        6.5|                 0.0|                 0.0|
| 26|2021-02-08|          6.5|        6.5|                 0.0|                 0.0|
| 26|2021-02-09|          6.5|        6.5|                 0.0|  

In [ ]:
df.cache()

In [ ]:
df.count()

In [ ]:
df.groupBy("sku").agg(countDistinct("currency")).show(70)

In [ ]:
1+1

In [ ]:
df.filter(F.col('sku')==464801).show()

+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|   sku|      date|sold_quantity|current_price|currency|listing_type|shipping_logistic_type|shipping_payment|minutes_active|
+------+----------+-------------+-------------+--------+------------+----------------------+----------------+--------------+
|464801|2021-02-01|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-02|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-03|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-04|            0|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|
|464801|2021-02-05|            1|       156.78|     REA|     classic|           fulfillment|   free_shipping|        1440.0|


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
feature_indexer = StringIndexer(inputCols = ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment'], 
                                outputCols = ['currency_indexed', 'listing_type_indexed', 'shipping_logistic_type_indexed', 'shipping_payment_indexed'])

onehotencoder = OneHotEncoder(dropLast=False, 
                              inputCols = ['currency_indexed', 'listing_type_indexed', 'shipping_logistic_type_indexed', 'shipping_payment_indexed'],
                              outputCols = ['currency_ohe', 'listing_type_ohe', 'shipping_logistic_type_ohe', 'shipping_payment_ohe'])

vector_assembler = VectorAssembler(inputCols = ['currency_ohe', 'listing_type_ohe', 'shipping_logistic_type_ohe', 'shipping_payment_ohe'],
                                   outputCol='features')

In [ ]:
pipeline = Pipeline(stages=[feature_indexer, onehotencoder, vector_assembler])
pipeline_model = pipeline.fit(df)
df_pipeline = pipeline_model.transform(df)

In [ ]:
df_pipeline.show(10)

In [ ]:
(df_pipeline
 .groupBy('sku') 
 .count()
 .show()
 .orderBy(F.col('count').desc()) 
 )

In [ ]:
df_indexed.show(3)

In [ ]:
train_df.groupBy('currency').count().show()

In [ ]:
train_df.groupBy('listing_type').count().show()

In [ ]:
train_df.groupBy('shipping_logistic_type').count().show()

In [ ]:
train_df.groupBy('shipping_payment').count().show()